# Visualisation de données géospatiales

## Bonnes pratiques appliquées aux feux de forêts

L'objectif de cet atelier est de pratiquer la datavisualisation en Python avec la librairie **plotly.express** en utilisant les jeux de données disponibles sur les incendies de forêts ainsi que l'indice d'humidité des sols.

**Francis Wolinski**

Consultant scientifique indépendant depuis 2013, bénéficiaire du programme résidentiel de **datacraft**
- Audit, Conseil et Projets en Data Science
- Formations professionnelles et enseignement de Python au niveau Master

Septembre 2022

In [ ]:
# imports
import glob

import numpy as np
import pandas as pd

import plotly.express as px

## 1. Jeux de données sur les incendies

### 1.1 BDIFF

**Base de Données sur les Incendies de Forêts en France**

> La base de données sur les incendies de forêt (BDIFF) est une application internet chargée de centraliser l'ensemble des données sur les incendies de forêt sur le territoire français depuis 2006 et de mettre l'ensemble de cette information à disposition du public et des services de l'Etat.
>
> L'application permet, à tout utilisateur intéressé, de consulter des données nationales, sur les feux de forêt jusqu'à l'année antérieure à l’année en cours et ce, agrégées à la commune.

https://bdiff.agriculture.gouv.fr/

In [ ]:
# load bdiff
bdiff = pd.read_csv("data/export_BDIFF_incendies_20220823.csv",
                    header=5,
                    sep=";")
bdiff.shape

In [ ]:
bdiff.head()

In [ ]:
# années
bdiff["Année"].unique()

In [ ]:
# départements
bdiff["Département"].unique()

### 1.2 Prométhée

**La banque de données sur les incendies de forêts en région Méditerranéenne en France**

In [ ]:
# https://www.promethee.com/incendies
promethee = pd.read_csv("data/liste_incendies_du_23_08_2022.csv",
                        header=2,
                        sep=";")
promethee.shape

In [ ]:
promethee.head()

In [ ]:
# années
promethee["Année"].unique()

In [ ]:
# départements
promethee["Département"].unique()

### 1.3 Comparaison

- Nombre d'enregistrements dans BDIFF correspondants à la zone de Prométhée
- Nombre d'enregistrements dans Prométhée correspondants à la période de BDIFF

In [ ]:
# nombre d'enregistrements dans BDIFF correspondants à la zone de Prométhée
len(bdiff.loc[bdiff["Département"].isin(promethee["Département"].unique())])

In [ ]:
# nombre d'enregistrements dans Prométhée correspondants à la période de BDIFF
len(promethee.loc[promethee["Année"].isin(bdiff["Année"].unique())])

## 2. Visualisations non cartographiques

### 2.1 Nombre d'incendies par an

**Pour BDIFF**

On affiche le nombre d'incendies par an avec la fonction `px.bar()`.

In [ ]:
# %load exos/exo_01.py

**Pour BDIFF et Prométhée**

On construit un dataframe aligné avec les données des 2 bases et on affiche en côte à côte le nombre d'incendies par an toujours avec la fonction `px.bar()`.

In [ ]:
# %load exos/exo_02.py

### 2.2 Surface brûlée par an

On crée une colonne dans chaque dataframe avec les surfaces en *ha*.

In [ ]:
# on passe les m2 en ha
bdiff["Surface brûlée (ha)"] = bdiff["Surface brûlée (m2)"]/10000
promethee["Surface parcourue (ha)"] = promethee["Surface parcourue (m2)"]/10000

**Pour BDIFF**

On affiche la surface brûlée en *ha* par an et la moyenne avec `px.bar()` et la méthode `add_hline()`.

In [ ]:
# %load exos/exo_03.py

On affiche les surfaces brûlées par an et la limite des mégafeux (1000 *ha* en Europe) avec la fonction `px.strip()` et la méthode `add_hline()`.

In [ ]:
# %load exos/exo_04.py

**Pour BDIFF et Prométhée**

On divise les données de surface brûlée de la base "Prométhée" par celle de "BDIFF" et on affiche le % correspondant à la surface brûlée en Méditerrannée par rapport à l'ensemble de la France avec la fontion `px.bar()` et la méthode `update_layout()` pour modifier le format de l'axe des Y.

In [ ]:
# %load exos/exo_05.py

**Pour BDIFF**

On affiche une carte proportionnelle *TreeMap* avec la surface brûlée par année, par département puis par commune avec la fonction `tree_map()`. Pourles communes, on prendra soin de les limiter aux 10 plus grands incendies par exemple.

In [ ]:
# %load exos/exo_06.py

## 3. Cartes sur les incendies de forêts

### 3.1 Cartes choroplèthes

Une carte *choroplèthe* (du grec χῶρος  : « zone/région » et πλῆθος  : « grand nombre, multitude ») est une carte thématique où les régions sont colorées ou remplies d'un motif qui montre une mesure statistique.

Pour produire une carte choroplèthe, il faut récupérer un fichier *geojson* de la partie du monde considérée. Ce fichier contient entre autres 2 informations importantes :
- Une clé qui désigne chaque sous-région,
- La description d'un polygone ou d'un multi-polygones sous la forme d'une liste de coordonnées (latitudes et longitudes),

Il faut ensuite faire correspondre les valeurs de la colonne du DataFrame qui contient les données relatives à chaque sous-région avec celles de la clé du fichier *geojson*.

Il est possible ensuite d'utiliser différents fonds de cartes et différents nuanciers.

**Fichier geojson**

Le fichier "./data/departements.geojson" utilisé ci-après provient du site : https://france-geojson.gregoiredavid.fr/ et il faut analyser le fichier pour trouver la clé correspondant à chaque sous-région.

In [ ]:
import json

with open("./data/departements.geojson") as f:
    departements = json.loads(f.read())

Analyse du fichier geojson :
- `departements["features"]` est une liste de 96 dictionnaires (les différents départements avec '2A' et '2B')
- chaque dictionnaire contient les clés :
    - 'type': 'Feature'
    - 'geometry': informations géométriques (polygones...)
    - 'properties' : informations d'identification qui permettent de référencer chaque dictionnaire

In [ ]:
# liste des valeurs de la clé "code"
codes = sorted([f["properties"]["code"] for f in departements["features"]])
print(*codes)

### 3.2 Carte du nombre d'incendies par année

On calcule le nombre d'incendies par année et par département et on affiche une carte avec la fonction `px.choropleth_mapbox()`.

In [ ]:
data1 = (bdiff.groupby(["Année", "Département"])
         .size()
         .reset_index()
         .rename(columns={0:"Nombre d'incendies"})
        )

data1.head()

In [ ]:
def display_year1(year):
    # select year
    df = data1.loc[data1["Année"]==year]
    
    # choropleth_mapbox
    return px.choropleth_mapbox(data_frame=df,
                                geojson=departements,
                                locations='Département',
                                color="Nombre d'incendies",
                                featureidkey='properties.code',
                                color_continuous_scale="RdYlGn_r",
                                mapbox_style="carto-positron",
                                zoom=4,
                                center={"lat": 47.0, "lon": 3.0},
                                opacity=0.5,
                                labels={"Nombre d'incendies": f"Nombre d'incendies en {year}"},
                                width=700)
    
fig = display_year1(2006)
fig

La carte est incomplète. Il faut rajouter dans "data1" les labels de l'ensemble des départements avec la méthode `reindex()` et en mettant les valeurs à *0*.

In [ ]:
# %load exos/exo_07.py

Exemple en utilisant les interactions apportées par la librairie **ipywidgets**.

In [ ]:
from ipywidgets import interact, widgets

interact(display_year1, year=widgets.IntSlider(min=2006, max=2021, step=1, value=2006));

Les échelles obtenues sont spécifiques à chacune des années. Si l'on souhaite féfinir une échelle absolue, il faut passer par la librairie plus complète **plotly.graph_objects** qui définit la classe `Choroplethmapbox` avec les attributs "zmin" et "zmax".

Les informations affichées via **plotly.express** doivent alors être spécifiées manuellement... On voit bien comment `plotly.express` simplifie la production de graphiques en unifiant les arguments de `plotly.graph_objects.Choroplethmapbox` et de la méthode `update_layout()`.

In [ ]:
import plotly.graph_objects as go

def display_year1(year):
    # select year and add all labels
    df = (data1.loc[data1["Année"]==year]
          .set_index("Département")
          .reindex(codes)
          .fillna(0)
          .reset_index()
         )
    
    # choropleth_mapbox
    fig = go.Figure(go.Choroplethmapbox(geojson=departements,
                                        locations=df['Département'],
                                        text=df['Département'],
                                        z=df["Nombre d'incendies"],
                                        featureidkey='properties.code',
                                        colorscale="RdYlGn_r",
                                        zmin=data1["Nombre d'incendies"].min(),
                                        zmax=data1["Nombre d'incendies"].max(),
                                        marker_opacity=0.5,
                                        ))
    fig.update_layout(mapbox_style="carto-positron",
                      mapbox_zoom=4,
                      mapbox_center={"lat": 47.0, "lon": 3.0},
                      width=700,
                      height=600)
    return fig
    
display_year1(2007)

### 3.3 Carte de la suface brûlée par année

On calcule dans "data2" la surface brûlée par année et par département et on l'affiche dans une carte avec la fonction `px.choropleth_mapbox()`.

In [ ]:
# %load exos/exo_08.py

### 3.4 Carte des surfaces incendiées par année

**Nuage de points géographiques** 

Pour obtenir la carte des surfaces incendiées, il faut obtenir la longitude et la latitude des communes où les incendies ont eu lieu.

On va utiliser le jeu de données des correspondances des codes Insee : https://public.opendatasoft.com/explore/dataset/correspondance-code-insee-code-postal/information/

On utilise uniquement les colonnes "Code INSEE", "Code Département" et "geo_point_2d" dont on extrait la latitude et la longitude de chaque commune avec la méthode `str.extract()` et une expression régulière. Ensuite on joint les 2 dataframes sur la colonne "Code INSEE".

Enfin, on utilise la fonction `px.scatter_mapbox()` qui permet d'afficher un nuage de points sur une carte.

In [ ]:
# load correspondance-code-insee-code-postal.csv
insee = pd.read_csv("data/correspondance-code-insee-code-postal.csv",
                   sep=";",
                   usecols=["Code INSEE", "Code Département", "geo_point_2d"])

insee.shape

In [ ]:
# %load exos/exo_09.py

### 3.5 Carte cummulative du nombre d'années avec incendies

On calule le nombre d'années avec incendies par département et on l'affiche dans une carte avec la fonction `px.choropleth_mapbox()`.

In [ ]:
# %load exos/exo_10.py

## 4. Humidité des sols

### 4.1 Indice d'humidité des sols

**Données mensuelles d'indice d'humidité des sols pour le dispositif CatNat.**

> L’indicateur SWI « Uniforme » : Le SWI (de l’anglais Soil Wetness Index ) est un indice d’humidité des sols documenté dans la littérature scientifique. Il représente, sur une profondeur d’environ deux mètres, l’état de la réserve en eau du sol par rapport à la réserve utile (eau disponible pour l’alimentation des plantes).

> Dans le modèle SIM, le territoire de France métropolitaine est découpé en mailles géographiques de 8 kilomètres de côté. Il est ainsi couvert par 8 981 mailles. Chacune des mailles ainsi définie est numérotée et recouvre tout ou partie d’une commune. Ce maillage est fixe et n’évolue pas d’une année sur l’autre. (coordonnées Lambert II étendu).

Source : https://donneespubliques.meteofrance.fr/?fond=produit&id_produit=301&id_rubrique=40

On charge l'ensemble des fichiers en convertissant la colonne "DATE" et en sélectionnant les années postérieures à 2006.

In [ ]:
# load SWI
dfs = []

for filename in glob.glob("data/secheresse/*.csv"):
    df = (pd.read_csv(filename, sep=";", decimal=",")
          .assign(DATE=lambda df_: pd.to_datetime(df_.DATE, format="%Y%m"))
          .loc[lambda df_: df_.DATE.dt.year >= 2006]
          )
    dfs.append(df)


swi = pd.concat(dfs)
swi.shape

In [ ]:
swi.head()

### 4.2 Lambert 93 vers GPS

Pour afficher ces informations dans une carte, il faut passer des coordonnées Lambert aux départements. On va tout d'abord passer des coordonnées Lambert aux coordonnées GPS, puis utiliser le fichier de l'Insee déjà utilisé pour passer des coordonnées GPS aux départements.

Contrairement à ce qui est indiqué dans la documentation, les coordonnées sont en Lambert93 (merci Julien <i class="fa fa-thumbs-up"></i>). Nous allons pouvoir les convertir en coordonnées GPS avec la librairie **lambert**.

In [ ]:
# imports

import matplotlib.pyplot as plt

from lambert import convertToWGS84Deg, Lambert93

In [ ]:
# scatterplot des coordonnées Lambert93
swi.plot.scatter(x="LAMBX", y="LAMBY", s=5);

On unicise les lignes pour ne calculer qu'une seule fois les coordonnées GPS car les données portent sur 16 années. Ensuite on calcule un "LambertPoint" pour chaque ligne avec la fonction `convertToWGS84Deg()` et on récupère la latitude et la longitude des points. Ensuite on recherche pour chacune des mailles la commune la plus proche géométriquement, via sa longitude et sa latitude, qui fournit ensuite le département

In [ ]:
# affectation du département correspondant à chaque maille
def get_departement(lon, lat):
    dist2 = (insee["lon"]-lon)**2 + (insee["lat"]-lat)**2
    return insee.loc[dist2.idxmin(), "Code Département"]

# conversion vers GPS puis obtention du département
swi_gps = (swi[["NUMERO", "LAMBX", "LAMBY"]].drop_duplicates()
           .assign(point=lambda df_: df_.apply(lambda x: convertToWGS84Deg(x[1], x[2], Lambert93), axis=1))
           .assign(lon=lambda df_: df_.point.apply(lambda x: x.getX()),
                   lat=lambda df_: df_.point.apply(lambda x: x.getY()))
           .reset_index(drop=True)
           .assign(code_departement=lambda df_: df_["lon"].combine(df_["lat"], get_departement))
           .sort_values(by="NUMERO")
          )

swi_gps.head()

On obtient bien une carte avec les données issues de SWI, cette fois-ci dans l'échelle GPS.

In [ ]:
# scatterplot des coordonnées GPS
swi_gps.plot.scatter(x="lon", y="lat", s=5);

### 4.3 Carte de l'humidité des sols

On joint le dataset SWI initial avec uniquement les départements obtenus par numéro de maille, on calcule la moyenne de l'indice *SWI* par département et par année (voir la classe `pd.Grouper()` pour utiliser directement la colonne "DATE" passée en index dans un `groupby()`) et on l'affiche dans une carte avec la fonction `px.choropleth_mapbox()`.

In [ ]:
# %load exos/exo_11.py

### 4.4 Calcul d'éventuelles corrélations entre les incendies et l'humidité des sols

Todo...

#### Article récent sur le sujet

*Will Norris*, **Using Panel and hvPlot to Visualize Decades of Wildfires**, *Towards Data Science*, September 2022.

https://towardsdatascience.com/using-panel-and-hvplot-to-visualize-decades-of-wildfires-2ac8d6f5db25

**Rappel**

- Librairies de dashboarding avec Python : Dash, Panel, Streamlit et Voilà.

- Librairies de visualisation avec Python : https://pyviz.org/index.html